In [1]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F

In [4]:
# 数据集类
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# 图片预处理
transform = transforms.Compose([
    transforms.Resize((222, 45)),
    transforms.ToTensor(),
])

In [12]:
from sklearn.model_selection import train_test_split

# 加载数据
data_dir = 'D:/Data/img/vv/训练/Output'  
images = []
labels = []
for file_name in os.listdir(data_dir):
    if file_name.endswith(('jpg', 'png', 'jpeg', 'gif')):
        images.append(os.path.join(data_dir, file_name))
        if '_1.' in file_name:
            labels.append(1)
        else:
            labels.append(0)

# 划分数据集
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.125, random_state=42) 

# 创建 DataLoader
train_dataset = CustomDataset(train_images, train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(val_images, val_labels, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = CustomDataset(test_images, test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# 定义网络
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 55 * 11, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 55 * 11)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# 创建模型
model = SimpleCNN()

In [8]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / (i+1)}')
    # 验证模型
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    print(f'[Epoch {epoch + 1}] validation loss: {val_loss / len(val_loader)}')

print('Finished Training')

[Epoch 1, Batch 1] loss: 0.09745520353317261
[Epoch 1, Batch 2] loss: 0.06131509132683277
[Epoch 1, Batch 3] loss: 0.06457571809490521
[Epoch 1, Batch 4] loss: 0.06538754981011152
[Epoch 1, Batch 5] loss: 0.06063253134489059
[Epoch 1, Batch 6] loss: 0.07131328682104747
[Epoch 1, Batch 7] loss: 0.0669017466051238
[Epoch 1, Batch 8] loss: 0.06148514337837696
[Epoch 1, Batch 9] loss: 0.06133467869626151
[Epoch 1] validation loss: 0.1256522610783577
[Epoch 2, Batch 1] loss: 0.04164174944162369
[Epoch 2, Batch 2] loss: 0.02909005619585514
[Epoch 2, Batch 3] loss: 0.03100167711575826
[Epoch 2, Batch 4] loss: 0.03393878135830164
[Epoch 2, Batch 5] loss: 0.03508103340864181
[Epoch 2, Batch 6] loss: 0.03136720244462291
[Epoch 2, Batch 7] loss: 0.030810094172401086
[Epoch 2, Batch 8] loss: 0.044029731187038124
[Epoch 2, Batch 9] loss: 0.05063949183871349
[Epoch 2] validation loss: 0.2547662742435932
[Epoch 3, Batch 1] loss: 0.009990459308028221
[Epoch 3, Batch 2] loss: 0.06788461934775114
[Epoch

In [11]:
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total}%')

Accuracy on the test images: 92.7710843373494%
